In [ ]:
import pandas as pd
import os
import json

In [ ]:
colocations = pd.read_csv("/network/projects/ecosystem-embeddings/SatBirdxsPlots/colocations_satbird_splot.csv")

In [ ]:
satbird = pd.read_csv("/network/projects/ecosystem-embeddings/SatBird_data_v2/USA_summer/all_summer_hotspots_final.csv")

In [ ]:
satbird = satbird.drop(columns=["Unnamed: 0.2", "Unnamed: 0.1", "Unnamed: 0"])

In [ ]:
satbird["bird"] = 1

In [ ]:
coloc = colocations[["hotspot_id", "sPlot_plot"]]

In [ ]:
coloc["plant"] = 1

In [ ]:
satbird = satbird.merge(coloc, how = "outer", left_on = "hotspot_id", right_on="hotspot_id")

In [ ]:
satbird["plant"] = satbird["plant"].fillna(0).astype(int)

In [ ]:
splots = pd.read_csv("/network/projects/ecosystem-embeddings/sPlotOpen/location_data.csv")

# GET DATA IN THE US

In [ ]:
from shapely.geometry import Point
import geopandas as gpd
import numpy as np

In [ ]:
# Load the continental U.S. boundary shapefile
# You can get this shapefile from Natural Earth or similar resources
gdf = gpd.read_file("/network/projects/ecosystem-embeddings/US_boundaries/cb_2018_us_nation_5m.shp")


In [ ]:
multi_poly = gdf["geometry"][0]

In [ ]:
polygons = list(multi_poly.geoms)

In [ ]:
from shapely.geometry import MultiPolygon, Polygon
import matplotlib.pyplot as plt


# Plot the MultiPolygon
fig, ax = plt.subplots()
for polygon in polygons:
    x, y = polygon.exterior.xy  # Get the exterior coordinates of each polygon
    ax.plot(x, y, color='blue')
    ax.fill(x, y, color='skyblue', alpha=0.5)  # Fill the polygon with transparency

ax.set_title("MultiPolygon Plot")
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.show()

In [ ]:
continental_us = polygons[np.argmax([p.area for p in polygons])]

In [ ]:
continental_us

In [ ]:

def is_in_us(latitude, longitude):
    point = Point(longitude, latitude)
    return continental_us.contains(point)


In [ ]:
def custom_operation(row):
    return (is_in_us(row["Latitude"], row["Longitude"]))

In [ ]:
splots["USA"] = splots.apply(custom_operation, axis = 1)

In [ ]:
splots_usa = splots[splots["USA"]]

In [ ]:
colocations

In [ ]:
plant_only = splots_usa[~splots_usa["PlotObservationID"].isin(colocations["sPlot_plot"])]

In [ ]:
plant_only["bird"] = 0

In [ ]:
plant_only = plant_only.rename(columns = {"Latitude":"lat", "Longitude":"lon", "PlotObservationID": "sPlot_plot"})

In [ ]:
len(plant_only), len(splots_usa)

In [ ]:
plant_only["split"] = "train"

In [ ]:
#ADD env data

In [ ]:
bioclim = pd.read_csv("/network/projects/ecosystem-embeddings/sPlotOpen/worldclim_data.csv")

In [ ]:
soilgrids = pd.read_csv("/network/projects/ecosystem-embeddings/sPlotOpen/soilgrid_data.csv")

In [ ]:
bioclim.columns = bioclim.columns.str.lower()
bioclim = bioclim.rename(columns = {"plotobservationid": "sPlot_plot"})
soilgrids.columns =soilgrids.columns.str.lower()
soilgrids = soilgrids.rename(columns = {"plotobservationid": "sPlot_plot"})


In [ ]:
merged_plants= plant_only.merge(soilgrids, left_on = "sPlot_plot", right_on = "sPlot_plot")

In [ ]:
merged_plants= merged_plants.merge(bioclim, left_on = "sPlot_plot", right_on = "sPlot_plot")

In [ ]:
merged_plants = merged_plants.drop(columns = ["Unnamed: 0", "unnamed: 0_x"])

In [ ]:
merged = satbird.merge(merged_plants, how = "outer")

In [ ]:
merged = merged.drop(columns = ["unnamed: 0_y", "USA"])

In [ ]:
merged.to_csv("/network/projects/ecosystem-embeddings/SatBirdxsPlots/satbird_splots.csv")

# save splits

In [ ]:
train = merged[merged["split"]=="train"]
valid = merged[merged["split"]=="valid"]
test = merged[merged["split"]=="test"]

In [ ]:
coloc_test = test[test["bird"]==1]
coloc_test = coloc_test[coloc_test["plant"]==1]

In [ ]:
coloc_test.to_csv("/network/projects/ecosystem-embeddings/SatBirdxsPlots/satbird_splots_test_split.csv")

In [ ]:
coloc_valid = valid[valid["bird"]==1]
coloc_valid = coloc_valid[coloc_valid["plant"]==1]

In [ ]:
coloc_valid.to_csv("/network/projects/ecosystem-embeddings/SatBirdxsPlots/satbird_splots_valid_split_coloc.csv")

In [ ]:
valid.to_csv("/network/projects/ecosystem-embeddings/SatBirdxsPlots/satbird_splots_valid_split.csv")

In [ ]:
len(coloc_valid), len(coloc_test), len(valid)

In [ ]:
train.to_csv("/network/projects/ecosystem-embeddings/SatBirdxsPlots/satbird_splots_train_split.csv")

In [ ]:
len(train)

Get present species only

In [22]:
from numpy import sort
import pickle
import pandas as pd
import numpy as np

def get_non_zero_indices(data_split):
    """
    This is for validation and test in the setup: satbirdxsplots
    :param data_split: 
    :return: 
    """
    with open("../data/sPlotOpen/sPlotOpen_targets_US_only.pkl", "rb") as pickle_file:
        targets = pickle.load(pickle_file)
    
    observations = []
    for plot_id in data_split['PlotObservationID'].tolist():
        target = targets.get(int(plot_id))
        observations.append(target)
    observations = np.array(observations)
    print(observations.shape)
    non_zero_indices = observations.sum(0) != 0
    num_classes = int(non_zero_indices.sum(0))
    print(num_classes)
    non_zero_indices = np.nonzero(observations.sum(axis=0))[0]
    return non_zero_indices

data_split = pd.read_csv("../data/satbirdxsplotopen/satbird_splots_test_split.csv")
non_zero_indices = get_non_zero_indices(data_split)
np.save('../data/satbirdxsplotopen/plant_test_species_indices_US_only_new.npy', non_zero_indices)
# original_non_zero_indices = sort(np.load('../data/satbirdxsplotopen/plant_test_species_indices_US_only.npy'))
# print(np.allclose(non_zero_indices, original_non_zero_indices))
print(non_zero_indices)
print(non_zero_indices.shape)

(248, 607)
502
[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  18
  19  20  21  22  24  25  26  27  28  29  30  31  32  33  34  35  36  37
  38  39  40  41  43  44  45  46  47  48  49  50  51  52  53  55  56  58
  59  60  61  62  63  64  65  66  67  68  69  71  72  73  74  75  76  77
  78  79  80  81  82  83  85  87  88  89  90  91  92  93  94  95  96  97
 101 103 104 105 106 107 108 109 110 111 112 113 115 116 117 118 119 120
 121 122 123 124 125 126 127 128 129 130 132 133 134 135 137 138 139 140
 141 142 143 144 146 148 149 151 152 153 154 155 156 158 160 161 162 164
 165 167 172 173 174 177 178 179 180 181 182 183 184 185 186 187 188 189
 190 191 192 193 194 195 196 197 198 202 204 205 206 207 209 210 213 214
 215 216 217 218 219 222 223 224 225 227 228 229 230 231 232 233 234 235
 237 238 239 240 241 242 243 244 245 246 248 249 250 251 253 256 258 259
 260 261 262 263 265 266 267 268 269 270 271 272 273 274 275 276 277 278
 279 280 281 282 283 284 288 289 290